# Esercitazione Guidata: Analisi di Dati con Python e Pandas

Benvenuti a questa esercitazione pratica! L'obiettivo di oggi è imparare a esplorare, pulire e visualizzare un dataset reale utilizzando le librerie Python che sono il pane quotidiano di ogni Data Analyst: **Pandas**, **NumPy** e **Matplotlib**.

Lavoreremo con il dataset `Dataset Abitudini Sportive`, che raccoglie informazioni sulle abitudini di allenamento di diverse persone. Impareremo a scoprire cosa si nasconde dietro i numeri e a rispondere a domande concrete basate sui dati.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Caricamento e Prima Ispezione del Dataset

In [ ]:
file_path = 'Dataset Abitudini Sportive.csv'
df = pd.read_csv(file_path)
df_originale = df.copy()

### `df.head()` e `df.info()`
Diamo una prima occhiata ai dati per avere un'idea delle colonne, dei tipi di dato e dei valori mancanti.

In [ ]:
print("Prime 5 righe:")
display(df.head())
print("\nInformazioni sul DataFrame:")
df.info()

## 2. Analisi Esplorativa e Individuazione di Anomalie

In [ ]:
df.describe()

#### Approfondimento: Identificare gli Outlier con il Metodo IQR

Un **outlier** è un valore che si discosta in modo anomalo dal resto dei dati. Può essere un errore di misurazione, un errore di inserimento dati (come i nostri `-999`), o un valore genuino ma molto raro. Gli outlier possono distorcere le nostre analisi statistiche (come la media) e i grafici.

Un metodo statistico robusto per identificarli è l'**Interquartile Range (IQR)**.

**Come funziona?**
1.  **Trova i Quartili:**
    *   **Primo Quartile (Q1):** Il valore al di sotto del quale si trova il 25% dei dati.
    *   **Terzo Quartile (Q3):** Il valore al di sotto del quale si trova il 75% dei dati.
2.  **Calcola l'IQR:**
    *   `IQR = Q3 - Q1`. Questo intervallo contiene il 50% centrale dei dati.
3.  **Definisci i "Baffi" (Whiskers):**
    *   **Limite Inferiore:** `Q1 - 1.5 * IQR`
    *   **Limite Superiore:** `Q3 + 1.5 * IQR`

Qualsiasi valore che cade al di fuori di questi limiti è considerato un outlier.

**Esempio Pratico:** Verifichiamo se i valori `-999` nella colonna `ore_settimanali_allenamento` vengono identificati come outlier.

In [ ]:
# Calcoliamo Q1, Q3 e IQR per la colonna originale
Q1 = df_originale['ore_settimanali_allenamento'].quantile(0.25)
Q3 = df_originale['ore_settimanali_allenamento'].quantile(0.75)
IQR = Q3 - Q1

# Definiamo i limiti
limite_inferiore = Q1 - 1.5 * IQR
limite_superiore = Q3 + 1.5 * IQR

print(f"Q1: {Q1}")
print(f"Q3: {Q3}")
print(f"IQR: {IQR}")
print(f"Limite Inferiore: {limite_inferiore}")
print(f"Limite Superiore: {limite_superiore}")

In [ ]:
# Filtriamo il DataFrame per trovare i valori che sono al di fuori dei limiti
outliers = df_originale[
    (df_originale['ore_settimanali_allenamento'] < limite_inferiore) | 
    (df_originale['ore_settimanali_allenamento'] > limite_superiore)
]

print("Outlier identificati:")
display(outliers)

**Conclusione:** Come possiamo vedere, il metodo IQR ha identificato correttamente i valori `-999` come outlier, poiché sono molto al di sotto del limite inferiore calcolato. Questo ci dà una giustificazione statistica, oltre che logica, per la loro rimozione o sostituzione.

### Esercizio: Trova gli outlier in `attrezzatura_comprata_annualmente`

Applica lo stesso metodo IQR alla colonna `attrezzatura_comprata_annualmente` del `df_originale` per vedere quali valori vengono classificati come anomali. Stampa a schermo i limiti e gli outlier trovati.

In [ ]:
# Il tuo codice qui


## 3. Pulizia dei Dati
Standardizziamo i valori anomali in `np.nan` e convertiamo le colonne in numeriche.

In [ ]:
df_pulito = df.copy()
df_pulito.replace([-999, '***', ''], np.nan, inplace=True)

# Identifichiamo le colonne categoriche per esclusione
colonne_categoriche = ['sesso', 'obiettivo_allenamento_label', 'frequenza_allenamento', 'motivazione_allenamento_label', 'tempo_riposo_settimanale_label']

for col in df_pulito.columns:
    if col not in colonne_categoriche:
        df_pulito[col] = pd.to_numeric(df_pulito[col], errors='coerce')

print("Valori mancanti dopo la standardizzazione e conversione:")
df_pulito.isnull().sum()

### Strategie per Gestire i Dati Mancanti
**Strategia 1: Eliminazione (`dropna`)**

In [ ]:
print(f"Forma originale: {df_pulito.shape}")
print(f"Forma dopo dropna(): {df_pulito.dropna().shape}")

**Strategia 2: Imputazione (Media/Mediana/Moda)**

In [ ]:
for colonna in df_pulito.columns:
    if df_pulito[colonna].isnull().any():
        if pd.api.types.is_numeric_dtype(df_pulito[colonna]):
            df_pulito[colonna].fillna(df_pulito[colonna].median(), inplace=True)
        else:
            df_pulito[colonna].fillna(df_pulito[colonna].mode()[0], inplace=True)
df_pulito.isnull().sum()

### Guida alla Scelta del Grafico Corretto

Prima di creare i grafici, è fondamentale capire *perché* scegliamo un tipo di grafico rispetto a un altro. La scelta dipende principalmente da due fattori:
1.  **Il tipo di dati** che vogliamo analizzare (quantitativi o categorici).
2.  **L'obiettivo dell'analisi**: cosa vogliamo scoprire o comunicare?

#### 1. Dati Quantitativi (Numerici)
- **Cosa sono?** Dati che rappresentano una quantità misurabile, come l'età, l'altezza, o le `ore_settimanali_allenamento`.
- **Obiettivo:** Capire la **distribuzione** dei dati. Vogliamo rispondere a domande come: "Quali sono i valori più comuni?", "I dati sono concentrati attorno a una media o sono sparsi?", "Ci sono picchi o anomalie?".
- **Grafico da usare:** **Istogramma (`Histogram`)**.
- **Perché?** Un istogramma raggruppa i dati in "intervalli" (o *bins*) e mostra quanti valori cadono in ciascun intervallo. Questo ci dà una visione immediata della forma della distribuzione, della sua tendenza centrale e della sua dispersione.

#### 2. Dati Categorici (Qualitativi)
- **Cosa sono?** Dati che rappresentano categorie o etichette, come `obiettivo_allenamento_label` (es. "Perdita peso", "Miglioramento muscolare") o `tempo_riposo_settimanale_label`.
- **Obiettivo:** Confrontare la **frequenza** o il conteggio di ciascuna categoria. Vogliamo rispondere a domande come: "Qual è la categoria più comune?", "Quante persone appartengono a ciascun gruppo?".
- **Grafico da usare:** **Grafico a Barre (`Bar Chart` o `Countplot`)**.
- **Perché?** Un grafico a barre assegna una barra a ogni categoria. L'altezza (o la lunghezza) della barra è proporzionale al numero di osservazioni in quella categoria, rendendo il confronto tra le diverse categorie semplice e immediato.

## 4. Visualizzazione: L'Impatto della Pulizia dei Dati

Ora che i nostri dati sono puliti, possiamo finalmente visualizzarli per capirne la struttura. Una delle pratiche più efficaci è confrontare i grafici **prima** e **dopo** la pulizia per vedere l'impatto del nostro lavoro.

### Esempio: Istogramma per `ore_settimanali_allenamento`

Analizziamo la distribuzione delle ore di allenamento. Creeremo due istogrammi affiancati: uno con i dati originali (`df_originale`) e uno con i dati puliti e imputati (`df_pulito`).

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Grafico 1: Dati Originali
# Convertiamo in numerico prima di plottare per evitare errori
dati_originali_numerici = pd.to_numeric(df_originale['ore_settimanali_allenamento'], errors='coerce')
ax1.hist(dati_originali_numerici.dropna(), bins=20, color='salmon', edgecolor='black')
ax1.set_title('Prima della Pulizia')
ax1.set_xlabel('Ore di Allenamento Settimanali')
ax1.set_ylabel('Frequenza')

# Grafico 2: Dati Puliti
ax2.hist(df_pulito['ore_settimanali_allenamento'], bins=20, color='skyblue', edgecolor='black')
ax2.set_title('Dopo la Pulizia e Imputazione')
ax2.set_xlabel('Ore di Allenamento Settimanali')
ax2.set_ylabel('Frequenza')

plt.suptitle('Impatto della Pulizia sulla Distribuzione delle Ore di Allenamento', fontsize=16)
plt.show()

**Domanda di Riflessione:** Osserva i due grafici. Quali differenze noti? In che modo i valori anomali (`-999`) nel dataset originale influenzavano la visualizzazione? Perché il secondo grafico è più utile per l'analisi?

### Esercizio: Visualizza l'impatto della pulizia su `attrezzatura_comprata_annualmente`

Ora tocca a te! Replica l'analisi precedente per la colonna `attrezzatura_comprata_annualmente`.

**Istruzioni:**
1.  Crea una figura con due subplot affiancati (`fig, (ax1, ax2) = ...`).
2.  Nel primo subplot (`ax1`), crea un istogramma della colonna `attrezzatura_comprata_annualmente` usando il DataFrame originale (`df_originale`).
3.  Nel secondo subplot (`ax2`), crea un istogramma della stessa colonna usando il DataFrame pulito (`df_pulito`).
4.  Aggiungi titoli appropriati a ciascun grafico e un titolo generale alla figura.

In [ ]:
# Il tuo codice qui


## 5. Analisi Guidata: Formulazione e Verifica di Ipotesi

Ora che i nostri dati sono puliti e abbiamo imparato a visualizzarli, possiamo passare alla fase più interessante: usare i dati per rispondere a delle domande. In questa sezione, formuleremo due ipotesi e useremo Pandas per verificarle.

### Ipotesi 1: Obiettivo di Allenamento e Spesa per Attrezzatura

**Ipotesi:** "Le persone che si allenano per la 'Perdita peso' acquistano in media meno attrezzatura (`attrezzatura_comprata_annualmente`) rispetto a quelle che puntano al 'Miglioramento muscolare'."

**Logica:** Questa ipotesi si basa sull'idea che l'allenamento per il miglioramento muscolare potrebbe richiedere una maggiore varietà di attrezzi (pesi, macchine, ecc.) rispetto a quello per la perdita di peso, che potrebbe essere più focalizzato su esercizi a corpo libero o cardio.

**Azione:** Per verificarlo, dobbiamo:
1.  Filtrare il DataFrame per isolare solo le righe che corrispondono ai due obiettivi: 'Perdita peso' e 'Miglioramento muscolare'.
2.  Raggruppare i dati per `obiettivo_allenamento_label`.
3.  Calcolare la media di `attrezzatura_comprata_annualmente` per ciascun gruppo.

In [ ]:
# 1. Filtriamo il DataFrame per i due gruppi di interesse
df_filtrato = df_pulito[df_pulito['obiettivo_allenamento_label'].isin(['Perdita peso', 'Miglioramento muscolare'])]

# 2. Raggruppiamo per obiettivo e calcoliamo la media della spesa per attrezzatura
media_spesa_per_obiettivo = df_filtrato.groupby('obiettivo_allenamento_label')['attrezzatura_comprata_annualmente'].mean()

print("Spesa media annuale per attrezzatura in base all'obiettivo:")
print(media_spesa_per_obiettivo)

#### Interpretazione Dettagliata dei Risultati

**Analisi del Risultato:**
Il codice che abbiamo eseguito ci ha restituito due valori numerici. Per il gruppo 'Miglioramento muscolare', la spesa media annuale è di circa 153.85 euro, mentre per il gruppo 'Perdita peso' è di circa 98.75 euro.

**Conclusione:**
Il risultato **conferma la nostra ipotesi iniziale**. I dati mostrano che, in media, le persone il cui obiettivo è il miglioramento muscolare spendono significativamente di più in attrezzatura rispetto a coloro che mirano alla perdita di peso (circa 55 euro in più all'anno). 

**Contesto Reale:**
Questa differenza suggerisce che esistono comportamenti di acquisto diversi legati a obiettivi di fitness differenti. Potrebbe implicare che chi si concentra sulla massa muscolare tende ad allestire una palestra domestica più fornita, acquistando pesi, manubri, bilancieri o macchinari specifici. Al contrario, chi punta a dimagrire potrebbe preferire attività che richiedono meno attrezzatura costosa, come la corsa, l'allenamento a corpo libero o l'utilizzo di app per il fitness. Questa informazione potrebbe essere molto utile, ad esempio, a un'azienda che vende articoli sportivi per calibrare le proprie campagne di marketing in modo più efficace, rivolgendosi a ciascun segmento di clientela con i prodotti più adatti.

### Ipotesi 2: Frequenza di Allenamento e Soddisfazione

**Ipotesi:** "Esiste una relazione tra la `frequenza_allenamento` e la `soddisfazione_allenamento`? In particolare, ci aspettiamo che a una maggiore frequenza di allenamento corrisponda una maggiore soddisfazione."

**Logica:** L'idea è che le persone che si allenano più spesso siano più motivate, vedano più risultati e, di conseguenza, riportino un livello di soddisfazione più alto.

**Azione:** Per investigare questa relazione, possiamo:
1.  Raggruppare il DataFrame per `frequenza_allenamento`.
2.  Calcolare la media della `soddisfazione_allenamento` per ogni livello di frequenza.
3.  Visualizzare il risultato con un grafico a barre per rendere la relazione più evidente.

In [ ]:
# 1. Raggruppiamo per frequenza e calcoliamo la soddisfazione media
soddisfazione_per_frequenza = df_pulito.groupby('frequenza_allenamento')['soddisfazione_allenamento'].mean().sort_index()

print("Soddisfazione media per frequenza di allenamento:")
print(soddisfazione_per_frequenza)

# 2. Visualizziamo i risultati per una migliore interpretazione
soddisfazione_per_frequenza.plot(kind='bar', figsize=(10, 6), color='c', edgecolor='black')
plt.title('Soddisfazione Media vs. Frequenza di Allenamento')
plt.xlabel('Frequenza di Allenamento')
plt.ylabel('Soddisfazione Media (scala 1-10)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')
plt.show()

#### Interpretazione Dettagliata dei Risultati

**Analisi del Risultato:**
L'output numerico e, in modo ancora più chiaro, il grafico a barre mostrano un andamento crescente. La soddisfazione media parte da un valore di circa 5.8 per chi si allena '1-2 volte a settimana', sale a 6.8 per chi si allena '3-4 volte a settimana' e raggiunge il picco di 8.0 per il gruppo '5-7 volte a settimana'.

**Conclusione:**
I dati **supportano fortemente la nostra ipotesi**. Esiste una chiara relazione positiva e quasi lineare tra la frequenza con cui una persona si allena e il suo livello di soddisfazione. Più spesso le persone si allenano, più tendono a essere soddisfatte del loro percorso di fitness.

**Contesto Reale:**
Questa scoperta evidenzia l'importanza della costanza nell'allenamento non solo per ottenere risultati fisici, ma anche per il benessere psicologico percepito. La soddisfazione potrebbe derivare da molteplici fattori legati alla frequenza: il rilascio di endorfine, una maggiore percezione dei progressi, il senso di disciplina e autoefficacia. Per un personal trainer o una palestra, questo dato è fondamentale: potrebbe suggerire che incentivare i clienti a mantenere una frequenza di allenamento costante (ad esempio, con programmi fedeltà o sfide) è una strategia chiave per aumentarne la soddisfazione e, di conseguenza, la fidelizzazione a lungo termine. In sostanza, un cliente più costante è un cliente più felice e più propenso a rimanere.

## Soluzioni

### Soluzione Esercizio: Trova gli outlier in `attrezzatura_comprata_annualmente`

In [ ]:
# Calcoliamo Q1, Q3 e IQR per la colonna
Q1_attrezzatura = df_originale['attrezzatura_comprata_annualmente'].quantile(0.25)
Q3_attrezzatura = df_originale['attrezzatura_comprata_annualmente'].quantile(0.75)
IQR_attrezzatura = Q3_attrezzatura - Q1_attrezzatura

# Definiamo i limiti
limite_inferiore_attrezzatura = Q1_attrezzatura - 1.5 * IQR_attrezzatura
limite_superiore_attrezzatura = Q3_attrezzatura + 1.5 * IQR_attrezzatura

print(f"Q1: {Q1_attrezzatura}")
print(f"Q3: {Q3_attrezzatura}")
print(f"IQR: {IQR_attrezzatura}")
print(f"Limite Inferiore: {limite_inferiore_attrezzatura}")
print(f"Limite Superiore: {limite_superiore_attrezzatura}")

# Filtriamo per trovare gli outlier
outliers_attrezzatura = df_originale[
    (df_originale['attrezzatura_comprata_annualmente'] < limite_inferiore_attrezzatura) | 
    (df_originale['attrezzatura_comprata_annualmente'] > limite_superiore_attrezzatura)
]

print("\nOutlier identificati in 'attrezzatura_comprata_annualmente':")
display(outliers_attrezzatura)

### Soluzione Esercizio: Visualizza l'impatto della pulizia su `attrezzatura_comprata_annualmente`

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Grafico 1: Dati Originali
# Convertiamo in numerico prima di plottare per evitare errori
dati_originali_numerici_attrezzatura = pd.to_numeric(df_originale['attrezzatura_comprata_annualmente'], errors='coerce')
ax1.hist(dati_originali_numerici_attrezzatura.dropna(), bins=20, color='salmon', edgecolor='black')
ax1.set_title('Prima della Pulizia')
ax1.set_xlabel('Attrezzatura Comprata Annualmente (€)')
ax1.set_ylabel('Frequenza')

# Grafico 2: Dati Puliti
ax2.hist(df_pulito['attrezzatura_comprata_annualmente'], bins=20, color='skyblue', edgecolor='black')
ax2.set_title('Dopo la Pulizia e Imputazione')
ax2.set_xlabel('Attrezzatura Comprata Annualmente (€)')
ax2.set_ylabel('Frequenza')

plt.suptitle('Impatto della Pulizia sulla Distribuzione della Spesa per Attrezzatura', fontsize=16)
plt.show()